In [22]:
!python -V

Python 3.7.3


In [23]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pycaret
from pycaret.regression import *

In [24]:
jan_tbl = pd.read_csv('data/jan.csv')
feb_tbl = pd.read_csv('data/feb.csv')

## check the shape of the jan dataframe

In [26]:
jan_tbl.shape

(1143691, 7)

In [27]:
jan_tbl.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'],
      dtype='object')

In [28]:
jan_tbl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143691 entries, 0 to 1143690
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   dispatching_base_num    1143691 non-null  object 
 1   pickup_datetime         1143691 non-null  object 
 2   dropOff_datetime        1143691 non-null  object 
 3   PUlocationID            267997 non-null   float64
 4   DOlocationID            1012291 non-null  float64
 5   SR_Flag                 0 non-null        float64
 6   Affiliated_base_number  1064376 non-null  object 
dtypes: float64(3), object(4)
memory usage: 61.1+ MB


## Compute duration from `pickup_datetime` and `dropOff_datetime` fields

In [29]:
jan_tbl['pickup_datetime'] = pd.to_datetime(jan_tbl['pickup_datetime'])
jan_tbl['dropOff_datetime'] = pd.to_datetime(jan_tbl['dropOff_datetime'])

In [30]:
jan_tbl['duration'] = jan_tbl['dropOff_datetime'] - jan_tbl['pickup_datetime']

jan_tbl['duration'] = jan_tbl.duration.apply(lambda td: td.total_seconds() / 60)

In [31]:
jan_tbl.describe().T

,count,mean,std,min,25%,50%,75%,max
PUlocationID,267997.0,134.015870,74.352860,1.000000,72.000000,132.000000,205.000000,265.0
DOlocationID,1012291.0,132.958111,80.774975,1.000000,62.000000,130.000000,212.000000,265.0
SR_Flag,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
duration,1143691.0,22.550803,469.699892,0.016667,8.433333,14.866667,25.633333,436365.0


In [32]:
feb_tbl['pickup_datetime'] = pd.to_datetime(feb_tbl['pickup_datetime'])
feb_tbl['dropOff_datetime'] = pd.to_datetime(feb_tbl['dropOff_datetime'])

In [33]:
feb_tbl['duration'] = feb_tbl['dropOff_datetime'] - feb_tbl['pickup_datetime']

feb_tbl['duration'] = feb_tbl.duration.apply(lambda td: td.total_seconds() / 60)

In [34]:
jan_tbl.describe().T

,count,mean,std,min,25%,50%,75%,max
PUlocationID,267997.0,134.015870,74.352860,1.000000,72.000000,132.000000,205.000000,265.0
DOlocationID,1012291.0,132.958111,80.774975,1.000000,62.000000,130.000000,212.000000,265.0
SR_Flag,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
duration,1143691.0,22.550803,469.699892,0.016667,8.433333,14.866667,25.633333,436365.0


In [35]:
jan_tbl.isnull().mean() * 100

dispatching_base_num        0.000000
pickup_datetime             0.000000
dropOff_datetime            0.000000
PUlocationID               76.567360
DOlocationID               11.489117
SR_Flag                   100.000000
Affiliated_base_number      6.935003
duration                    0.000000
dtype: float64

In [36]:
jan_tbl.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2022-01-01 00:31:00+00:00,2022-01-01 01:05:00+00:00,NaN,NaN,NaN,B00009,34.000000
1,B00009,2022-01-01 00:37:00+00:00,2022-01-01 01:05:00+00:00,NaN,NaN,NaN,B00009,28.000000
2,B00037,2022-01-01 00:56:37+00:00,2022-01-01 01:06:11+00:00,NaN,85.0,NaN,B00037,9.566667
3,B00037,2022-01-01 00:19:54+00:00,2022-01-01 00:30:47+00:00,NaN,85.0,NaN,B00037,10.883333
4,B00037,2022-01-01 00:41:49+00:00,2022-01-01 00:52:16+00:00,NaN,188.0,NaN,B00037,10.450000


## Modeling with `pycaret`

In [40]:
setup(jan_tbl, target='duration', ignore_features= ['pickup_datetime', 'dropOff_datetime', 'SR_Flag', 'dispatching_base_num', 'Affiliated_base_number' ] )

best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,15.6232,74188.1171,177.8005,-0.0001,0.9097,4.6935,0.1080
omp,Orthogonal Matching Pursuit,15.6216,74188.0132,177.7996,-0.0001,0.9097,4.7034,0.0820
br,Bayesian Ridge,15.6203,74188.0562,177.7995,-0.0001,0.9098,4.7040,0.1540
en,Elastic Net,15.6235,74188.1228,177.8006,-0.0001,0.9097,4.6927,0.0960
lar,Least Angle Regression,15.6238,74188.1268,177.8007,-0.0002,0.9097,4.6918,0.0890
llar,Lasso Least Angle Regression,15.6218,74188.7943,177.8022,-0.0002,0.9105,4.7458,0.0830
dummy,Dummy Regressor,15.6218,74188.7902,177.8022,-0.0002,0.9105,4.7458,0.0880
ridge,Ridge Regression,15.6238,74188.1168,177.8007,-0.0002,0.9097,4.6918,0.0770
lr,Linear Regression,15.6238,74188.1168,177.8007,-0.0002,0.9097,4.6918,1.3970
huber,Huber Regressor,14.0668,74228.4751,178.0580,-0.0112,0.8239,3.3152,2.6100


In [45]:
evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [46]:
prediction = predict_model(best_model, feb_tbl)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Lasso Regression,16.4623,8220.7689,90.6685,-0.0005,0.9084,4.5134
